In [14]:
import merkletools

In [15]:
mt = merkletools.MerkleTools(hash_type="keccak_256")

mt.add_leaf('0000000000000000000000000000000000000000000000000000000000000017', True)
mt.add_leaf(["bitcoin", "blockchain"], True)

mt.make_tree()

In [47]:
leaf_value =  mt.get_leaf(0)

In [19]:
mt.get_proof(1)

[{'left': 'c737aa53bf5c1c135465bab4dc4051f2a6123e709fa2d8b349bd85a5617491ca'},
 {'right': '7ee156df5091fbef71b96557542210a9c9ca851cc85aaf60026519b4aaccf491'}]

In [20]:
mt.get_leaf(1)

'7dee6e1aa550de37364ec77e03e62ea56bf42037b8297280de9d844d88444e4d'

In [21]:
mt.get_merkle_root()

'b2f0283f3d9e254bd18e12ef6da55e5c2373c2577e5e07f3157aa608d58443d8'

In [12]:
import sha3
from eth_abi.abi import encode_abi

sha3.keccak_256(encode_abi(['int32'], [23])).hexdigest()

##sha3.keccak_256(bytearray.fromhex('0000000000000000000000000000000000000000000000000000000000000017')).digest()

'c624b66cc0138b8fabc209247f72d758e1cf3343756d543badbf24212bed8c15'

In [6]:
import numpy as np

In [21]:
arr = np.array([1, 2, 150], dtype=np.int32)

In [24]:
hex(arr[1])

'0x2'

In [28]:
from web3 import Web3
from solcx import compile_files, install_solc

In [29]:
install_solc()

Version('0.8.15')

In [38]:
contract_id, compiled_sol = compile_files(["verifyMerkleProof.sol"], output_values=['abi', 'bin']).popitem()

In [39]:
w3 = Web3(Web3.EthereumTesterProvider())

In [40]:
verifyMerkle = w3.eth.contract(abi=compiled_sol['abi'], bytecode=compiled_sol['bin'])

In [42]:
tx_hash = verifyMerkle.constructor().transact()

In [43]:
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

In [44]:
merkleContract = w3.eth.contract(address=tx_receipt.contractAddress, abi=compiled_sol['abi'])

In [56]:
def proof_translate(mt_ins, x):
    root = '0x' + mt_ins.get_merkle_root()
    leaf = '0x' + mt_ins.get_leaf(x)
    res_proof = []
    res_bool = []
    for p in mt_ins.get_proof(x):
        if list(p.keys())[0] == 'left':
            res_proof.append('0x' + p['left'])
            res_bool.append(False)
        else:
            res_proof.append('0x' + p['right'])
            res_bool.append(True)
    return root, leaf, res_proof, res_bool


In [59]:
merkleContract.functions.verify(*proof_translate(mt, 1)).call()

True

In [63]:
encode_abi(['(int32,int32)'], [(23, 12)])

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x17\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0c'

In [41]:
w3.eth.get_balance(acct.address)

0